In [1]:
# coding: utf-8

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module
from PIL import Image
from sklearn.model_selection import train_test_split
import glob
import os
import csv

PATH = os.getcwd()
image_size=36

#seanセットフォルダ一覧取得
datadir = PATH + "\\dataset_h"
traindir = datadir + "\\train"
testdir = datadir + "\\test"
train_seanset = glob.glob(traindir + "\\*")
test_seanset = glob.glob(testdir + "\\*")

train_sean_seqs = []
test_sean_seqs = []
for sean in train_seanset:
    #sean_imgcfile = glob.glob(sean + "\\object*.png")[0]
    sean_imgdfile = glob.glob(sean + "\\objectd*.png")[0]
    s_datafiles = glob.glob(sean + "\\data\\*.csv")
    s_labelfiles = glob.glob(sean + "\\label\\*.csv")
    for t in range(len(s_datafiles)):
        pair = []
        pair.append(s_datafiles[t])
        pair.append(s_labelfiles[t])
        pair.append(sean_imgdfile)
        train_sean_seqs.append(pair) #dataとlabelと画像名をセットで登録
for sean in test_seanset:
    #sean_imgcfile = glob.glob(sean + "\\object*.png")[0]
    sean_imgdfile = glob.glob(sean + "\\objectd*.png")[0]
    s_datafiles = glob.glob(sean + "\\data\\*.csv")
    s_labelfiles = glob.glob(sean + "\\label\\*.csv")
    for t in range(len(s_datafiles)):
        pair = []
        pair.append(s_datafiles[t])
        pair.append(s_labelfiles[t])
        pair.append(sean_imgdfile)
        test_sean_seqs.append(pair) #dataとlabelと画像名をセットで登録

img_skeleton_sets_train = []
img_skeleton_sets_test = []

def make_datasets(sean_seqs):
    img_skeleton_sets = []
    for i in range(len(sean_seqs)):
        #画像読み込み・リサイズ・配列に変換
        image = Image.open(sean_seqs[i][2])
        image = image.resize((image_size, image_size))
        img_array = np.asarray(image)
        #画像マスク作成(元画像の画素値が外れ値(0～500)の部分を1にする、他は0)
        img_mask_array = np.zeros((image_size, image_size), np.uint8)
        for h in range(img_array.shape[0]):
            for w in range(img_array.shape[1]):
                if 0 <= img_array[h,w] < 500:
                    img_mask_array[h,w] = 1
        data_points = []
        data_masks = []
        label_points = []
        label_masks = []
        #学習用骨格データ読み込み
        with open(sean_seqs[i][0]) as f:
            reader = csv.reader(f)
            num = 0
            for row in reader:
                if num == 0:
                    for point in row:
                        if int(point) == -10000:
                            data_points.append(float(0))
                            data_masks.append(0)
                        else:
                            data_points.append(float(point))
                            data_masks.append(1)
                    data_points = np.asarray(data_points)
                    data_masks = np.asarray(data_masks)
                    num += 1
        #ラベル用骨格データ読み込み
        with open(sean_seqs[i][1]) as f:
            reader = csv.reader(f)
            num = 0
            for row in reader:
                if num == 0:
                    for point in row:
                        if int(point) == -10000:
                            label_points.append(float(0))
                            label_masks.append(0)
                        else:
                            label_points.append(float(point))
                            label_masks.append(1)
                    label_points = np.asarray(label_points)
                    label_masks = np.asarray(label_masks)
                    num += 1
        #画像・学習用・ラベル用スケルトンデータをパッケージング
        img_skeleton_pack = []
        img_skeleton_pack.append(img_array)
        img_skeleton_pack.append(img_mask_array)
        img_skeleton_pack.append(data_points)
        img_skeleton_pack.append(data_masks)
        img_skeleton_pack.append(label_points)
        img_skeleton_pack.append(label_masks)
        #パッケージをデータセットに登録
        img_skeleton_sets.append(img_skeleton_pack)
    return img_skeleton_sets

img_skeleton_sets_train = np.array(make_datasets(train_sean_seqs))
img_skeleton_sets_test = np.array(make_datasets(test_sean_seqs))
print("fin loading")

fin loading


In [2]:
# 数合わせ用
img_skeleton_sets_train, govage = train_test_split(
    img_skeleton_sets_train,
    shuffle = False,
    train_size = 495
)
img_skeleton_sets_test, govage = train_test_split(
    img_skeleton_sets_test,
    shuffle = False,
    train_size = 120
)
#print(img_skeleton_sets_train.shape, img_skeleton_sets_test.shape, govage.shape)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [17]:
#print("[0]image     ", img_skeleton_sets_train[0][0].shape, "\n", img_skeleton_sets_train[0][0]) #画像
#print("[1]image-mask", img_skeleton_sets_train[0][1].shape, "\n", img_skeleton_sets_train[0][1]) #画像マスク
#print("[2]input     ", img_skeleton_sets_train[0][2].shape, "\n", img_skeleton_sets_train[0][2]) #骨格(入力)
#print("[3]input-mask", img_skeleton_sets_train[0][3].shape, "\n", img_skeleton_sets_train[0][3]) #骨格マスク(入力)
#print("[4]label     ", img_skeleton_sets_train[0][4].shape, "\n", img_skeleton_sets_train[0][4]) #骨格(ラベル)
#print("[5]label-mask", img_skeleton_sets_train[0][5].shape, "\n", img_skeleton_sets_train[0][5]) #骨格マスク(ラベル)

print(type(img_skeleton_sets_train))
#print(img_skeleton_sets_train[0].shape)
print(img_skeleton_sets_train.shape, "img_skeleton_sets_train shape")
print(img_skeleton_sets_test.shape, "img_skeleton_sets_test  shape")

<class 'numpy.ndarray'>
(495, 6) img_skeleton_sets_train shape
(120, 6) img_skeleton_sets_test  shape


In [3]:
data_train = []
img_train = []
mask_train = []
imask_train = []
data_test = []
img_test = []
mask_test = []
imask_test = []
label_train = []
lmask_train = []
label_test = []
lmask_test = []
for i in range(len(img_skeleton_sets_train)):
    data_train.append(img_skeleton_sets_train[i][2])
    img_train.append(img_skeleton_sets_train[i][0])
    mask_train.append(img_skeleton_sets_train[i][3])
    imask_train.append(img_skeleton_sets_train[i][1])
    label_train.append(img_skeleton_sets_train[i][4])
    lmask_train.append(img_skeleton_sets_train[i][5])
for i in range(len(img_skeleton_sets_test)):
    data_test.append(img_skeleton_sets_test[i][2])
    img_test.append(img_skeleton_sets_test[i][0])
    mask_test.append(img_skeleton_sets_test[i][3])
    imask_test.append(img_skeleton_sets_test[i][1])
    label_test.append(img_skeleton_sets_test[i][4])
    lmask_test.append(img_skeleton_sets_test[i][5])
    
print(data_train[0])
print(img_train[0])
print(mask_train[0])
print(imask_train[0])
print(label_train[0])
print(lmask_train[0])

[-267. -214.  151.    0.    0.    0. -229. -102.  -87. -228.  -74.  -66.
    0.    0.    0. -247. -151.  192. -249. -108.  208. -224.  -48.   -8.
 -225.  -33.  -62. -247. -145.  180.    0.    0.    0. -225.  -42.  -62.
 -234.  -34.  -73. -250. -145.  180. -224.  -73.  -36. -231.  -49.  -79.
 -239.  -40.  -96. -225.  -97.  -60. -228.  -74.  -66. -235.  -56.  -96.
 -238.  -51. -105.    0.]
[[3425 3425 3429 ... 3441 3441 3441]
 [3416 3416 3416 ... 3418 3418 3419]
 [3415 3415 3412 ... 3399 3399 3396]
 ...
 [3028 3028 3024 ... 2922 2922 2954]
 [3015 3015 3014 ... 2927 2927 2945]
 [3015 3015 3014 ... 2927 2927 2945]]
[1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[-140.  -46. -418. -113.  -59. -423.    0.    0.    0.    0.    0.    0.
    0.    0.    0.  -84.  -39. -437.  -58.  -28. -435.  -

In [4]:
data_trainF = np.array(data_train).astype('float32')/100
data_testF = np.array(data_test).astype('float32')/100
img_trainF = np.array(img_train).astype('float32')/10000
img_testF = np.array(img_test).astype('float32')/10000
mask_trainF = np.array(mask_train).astype('float32')
mask_testF = np.array(mask_test).astype('float32')
imask_trainF = np.array(imask_train).astype('float32')
imask_testF = np.array(imask_test).astype('float32')
label_trainF = np.array(label_train).astype('float32')/100
label_testF = np.array(label_test).astype('float32')/100
lmask_trainF = np.array(lmask_train).astype('float32')
lmask_testF = np.array(lmask_test).astype('float32')

In [5]:
print("data        train ",data_trainF.shape)
print("data        test  ",data_testF.shape)
print("image       train ",img_trainF.shape)
print("image       test  ",img_testF.shape)
print("data-mask   train ",mask_trainF.shape)
print("data-mask   test  ",mask_testF.shape)
print("image-mask  train ",imask_trainF.shape)
print("image-mask  test  ",imask_testF.shape)
print("label       train ",label_trainF.shape)
print("label       test  ",label_testF.shape)
print("label-mask  train ",lmask_trainF.shape)
print("label-mask  test  ",lmask_testF.shape)

data        train  (495, 64)
data        test   (120, 64)
image       train  (495, 36, 36)
image       test   (120, 36, 36)
data-mask   train  (495, 64)
data-mask   test   (120, 64)
image-mask  train  (495, 36, 36)
image-mask  test   (120, 36, 36)
label       train  (495, 63)
label       test   (120, 63)
label-mask  train  (495, 63)
label-mask  test   (120, 63)


In [6]:
trans = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()]#,torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
)

In [7]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, datas, labels, img_array, data_masks, label_masks, img_masks, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.label = labels
        self.img_array = img_array
        self.masks = data_masks
        self.lmasks = label_masks
        self.imasks = img_masks

        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data[idx]
        i_label = self.label[idx]
        i_img = self.img_array[idx]
        i_mask = self.masks[idx]
        i_lmask = self.lmasks[idx]
        i_imask = self.imasks[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_label = np.array(i_label.astype(np.float32))
        out_data = np.array(i_data.astype(np.float32))
        out_img = np.array(i_img.astype(np.float32))
        out_mask = np.array(i_mask.astype(np.float32))
        out_lmask = np.array(i_lmask.astype(np.float32))
        out_imask = np.array(i_imask.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            #out_data = self.transform(i_data)
            out_img = self.transform(out_img)
            out_imask = self.transform(out_imask)
            #out_label = self.transform2(out_label)
        
        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)

        return out_data, out_label, out_img, out_mask, out_lmask, out_imask
        #return batch_set

In [8]:
trainset = Mydatasets(datas = data_trainF, labels = label_trainF, img_array = img_trainF, data_masks = mask_trainF, label_masks = lmask_trainF, img_masks = imask_trainF, transform = trans)
testset = Mydatasets(datas = data_testF, labels = label_testF, img_array = img_testF, data_masks = mask_testF, label_masks = lmask_testF, img_masks = imask_testF, transform = trans)

In [9]:
BATCH_SIZE = 15
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE,
                        shuffle = True, num_workers = 0)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)

In [10]:
class Ignore(nn.Module):
    def __init__(self, input_dim, output_dim, activation = lambda x: x):
        '''
        引数:
            input_dim: 入力次元
            output_dim: 出力次元
            activation: 活性化関数
        パラメータ:
            W: 重み
            b: バイアス
        '''
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(np.random.normal(size = (output_dim, input_dim))))
        self.b = nn.Parameter(torch.Tensor(np.zeros(output_dim)))
        self.activation = activation
    def forward(self, x, mask):
        
        out = torch.empty_like(x).to(torch.device("cuda:0"))
        masked_x = torch.mul(x, mask)
        
        try:
            m_size = torch.Tensor(mask.size()[1]).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask, 1).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            for b in range(out.size()[0]):
                out[b] = torch.add(torch.mul(rate[b], torch.sum(torch.mul(masked_x[b], self.W), 1)), self.b)
        except IndexError:
            m_size = torch.Tensor(mask.size()).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            out = torch.add(torch.mul(rate, torch.sum(torch.mul(masked_x, self.W))), self.b)
        
        return self.activation(out)

In [11]:
import scipy.stats as stats

In [12]:
class HandGraspRecolletion(nn.Module):
    def __init__(self):
        super(HandGraspRecolletion, self).__init__()
        #CNN
        self.pool = nn.AvgPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        self.bn2d1 = nn.BatchNorm2d(16)
        self.bn2d2 = nn.BatchNorm2d(32)
        
        self.fcm = Ignore(32 * 7 * 7 + 64, 32 * 7 * 7 + 64)
        self.bnm = nn.BatchNorm1d(32 * 7 * 7 + 64)
        #fully connect for hand Location(x,y,z)
        self.fcL1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcL2 = nn.Linear(300, 30)
        self.fcL3 = nn.Linear(30, 3)        
        self.bnL1 = nn.BatchNorm1d(300)
        self.bnL2 = nn.BatchNorm1d(30)
        #fully connect for hand Pose Descriptor(8 properties)
        self.fcPD1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcPD2 = nn.Linear(300, 60)
        self.fcPD3 = nn.Linear(60, 8)
        self.bnPD1 = nn.BatchNorm1d(300)
        self.bnPD2 = nn.BatchNorm1d(60)
        
        #Autoencoder
        self.mask = Ignore(84, 84)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.dense_enc1 = nn.Linear(84, 40)
        self.bn1 = nn.BatchNorm1d(40)
        self.dense_enc2 = nn.Linear(40, 18)
        self.bn2 = nn.BatchNorm1d(18)
        self.dense_enc3 = nn.Linear(18,8)
    
        self.dense_dec1 = nn.Linear(8,16)
        self.bn4 = nn.BatchNorm1d(16)
        self.dense_dec2 = nn.Linear(16, 32)
        self.bn5 = nn.BatchNorm1d(32)
        self.drop1 = nn.Dropout(p=0.2)
        self.dense_dec3 = nn.Linear(32, 63)

    def cnn_part(self, x, y, m, im):
        #Conv2d 1
        x = self.conv1(x)
        im = self.conv1(im)
        im_conv1 = im.clone().detach().to(torch.device("cuda:0"))
        for c in range(im.size()[1]):
            mode, ind = torch.mode(torch.reshape(im[0][c].clone().detach(), (-1, im.size()[2] * im.size()[3])))
            im_conv1[:,c] = torch.sub(im_conv1[:,c], mode.item())
        #ReLU→BatchNorm2d 1
        x = self.bn2d1(self.relu(x))
        #AvgPool2d
        x = self.pool(x)
        im = self.pool(im_conv1)
        #Conv2d 2
        x = self.conv2(x)
        im = self.conv2(im)
        im_conv2 = im.clone().detach().to(torch.device("cuda:0"))
        for c in range(im.size()[1]):
            mode, ind = torch.mode(torch.reshape(im[0][c].clone().detach(), (-1, im.size()[2] * im.size()[3])))
            im_conv2[:,c] = torch.sub(im_conv2[:,c], mode.item())
        #ReLU→BatchNorm2d 2
        x = self.bn2d2(self.relu(x))
        #AvgPool2d
        x = self.pool(x)
        im = self.pool(im_conv2)
        #1次元ベクトル化
        x = x.view(x.size()[0], -1)
        im = im.view(im.size()[0], -1)
        #マスク再構築(0→1, 0以外→0)
        im = torch.logical_not(torch.logical_and(im, torch.tensor([True]).to(torch.device("cuda:0")))).float()
        #骨格データと結合
        x = torch.cat([x, y], axis = -1)
        m = torch.cat([im, m], axis = -1)
        #MaskedLinear
        x = self.fcm(x, m)
        x = self.bnm(self.relu(x))
        xL = self.fcL1(x)
        xPD = self.fcPD1(x)
        xL = self.bnL1(self.relu(xL))
        xPD = self.bnPD1(self.relu(xPD))
        xL = self.fcL2(xL)
        xPD = self.fcPD2(xPD)
        xL = self.bnL2(self.relu(xL))
        xPD = self.bnPD2(self.relu(xPD))
        xL = self.fcL3(xL)
        xPD = self.fcPD3(xPD)
        return xL, xPD
        
    def encoder(self, x):
        #x = torch.div(x, 100.)
        #x = self.mask(x, m)
        x = self.dense_enc1(x)
        x = self.bn1(self.relu(x))
        x = self.dense_enc2(x)
        x = self.bn2(self.relu(x))
        x = self.dense_enc3(x)
        return x

    def decoder(self, x):
        x = self.dense_dec1(x)
        x = self.bn4(self.relu(x))
        x = self.dense_dec2(x)
        x = self.bn5(self.relu(x))
        #x = self.drop1(x)
        x = self.dense_dec3(x)
        #x = self.mask(x, m)
        #x = torch.mul(x, 100.)
        return x
    
    def forward(self, x, y, m, im):
        xLoc, xPD = self.cnn_part(x, y, m, im)
        xPose = self.decoder(xPD)
        return xLoc, xPose, xPD
    #    if x != None and z == None:
    #        z = self.encoder(x)
    #        x = self.decoder(z)
    #    elif x == None and z != None:
    #        print("decoder only")
    #        x = self.decoder(z)
    #    return x, z

    #def forward(self, x=None, z=None):
    #    if x != None and z == None:
    #        z = self.encoder(x)
    #        x = self.decoder(z)
    #    elif x == None and z != None:
    #        print("decoder only")
    #        x = self.decoder(z)
    #    return x, z

In [13]:
class MaskMSELoss(nn.Module):

    def __init__(self):
        super(MaskMSELoss, self).__init__()
        #self.margin = margin

    def forward(self, inputs, labels, lmasks):
        m_size = lmasks.size()[1]
        m_sum = torch.sum(lmasks, 1)
        #print("m_size:",m_size)
        #print("m_sum:",m_sum)
        rate = (m_size / m_sum).reshape(BATCH_SIZE,-1)
        #print("rate:",rate)
        for i in range(rate.size()[0]):
            #print(rate[i].item())
            if np.isinf(rate[i].item()):
                #print("inf!")
                rate[i] = 0.
        #print("rate+:", rate)
        masked_in = torch.mul(inputs, lmasks)
        masked_lb = torch.mul(labels, lmasks)
        
        #loss = torch.sub(masked_in, masked_lb)
        #print("sub:",loss)
        #loss = torch.pow(loss, 2)
        #print("pow:",loss)
        #loss = torch.sum(loss, 1)
        #print("sum:",loss)
        #loss = torch.div(loss, m_sum)
        #print("div:",loss)
        #loss = torch.mul(rate, loss)
        #print("gain2:",loss)
        #loss = torch.mean(loss)
        #print("mean:",loss)
        
        loss = torch.sub(masked_in, masked_lb)
        loss = torch.pow(loss, 2)
        loss = torch.mul(loss, rate)
        loss = torch.mean(loss, 1)
        loss = torch.mean(loss)
        
        return loss

In [14]:
LEARNING_RATE = 0.01
WEIGHT_DECAY = 0.0001
MOMENTUM = 0.9

device = torch.device("cuda:0")
#net = Net().to(device)
#ae = Autoencoder()
net = HandGraspRecolletion().to(device)
net.load_state_dict(torch.load(PATH + "\\hand_AE_model"))
for p in net.parameters(): # freeze all model parameters
    p.requires_grad = False
for p in net.conv1.parameters():
    p.requires_grad = True
for p in net.conv2.parameters():
    p.requires_grad = True
for p in net.bn2d1.parameters():
    p.requires_grad = True
for p in net.bn2d2.parameters():
    p.requires_grad = True
for p in net.fcm.parameters():
    p.requires_grad = True
for p in net.bnm.parameters():
    p.requires_grad = True
for p in net.fcL1.parameters():
    p.requires_grad = True
for p in net.fcL2.parameters():
    p.requires_grad = True
for p in net.fcL3.parameters():
    p.requires_grad = True
for p in net.bnL1.parameters():
    p.requires_grad = True
for p in net.bnL2.parameters():
    p.requires_grad = True
for p in net.fcPD1.parameters():
    p.requires_grad = True
for p in net.fcPD2.parameters():
    p.requires_grad = True
for p in net.fcPD3.parameters():
    p.requires_grad = True
for p in net.bnPD1.parameters():
    p.requires_grad = True
for p in net.bnPD2.parameters():
    p.requires_grad = True    
criterion = MaskMSELoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

train_loss_value=[]      #trainingのlossを保持するlist
test_loss_value=[]       #testのlossを保持するlist

train_input_value = []
train_output_value = []
train_desc_value = []
train_handloc_value = []
test_input_value = []
test_output_value = []
test_desc_value = []
test_handloc_value = []

train_size = data_trainF.shape[0]
test_size = data_testF.shape[0]

max_train_loss_value = 0.
max_test_loss_value = 0.
min_test_loss_value = None

In [109]:
keys = ["conv1.weight","conv1.bias","conv2.weight","conv2.bias","bn2d1.weight","bn2d1.bias","bn2d2.weight",\
        "bn2d2.bias","fcm.W","fcm.b","bnm.weight","bnm.bias","fcL1.weight","fcL1.bias","fcL2.weight",\
        "fcL2.bias","fcL3.weight","fcL3.bias","bnL1.weight","bnL1.bias","bnL2.weight","bnL2.bias",\
        "fcPD1.weight","fcPD1.bias","fcPD2.weight","fcPD2.bias","fcPD3.weight","fcPD3.bias","bnPD1.weight",\
        "bnPD1.bias","bnPD2.weight","bnPD2.bias","mask.W","mask.b","dense_enc1.weight","dense_enc1.bias",\
        "bn1.weight","bn1.bias","dense_enc2.weight","dense_enc2.bias","bn2.weight","bn2.bias",\
        "dense_enc3.weight","dense_enc3.bias","dense_dec1.weight","dense_dec1.bias","bn4.weight","bn4.bias",\
        "dense_dec2.weight","dense_dec2.bias","bn5.weight","bn5.bias","dense_dec3.weight","dense_dec3.bias"]
#for k, p in zip(keys, net.parameters()):
#    print(k, "\n", p, "\n")
#for k, p in zip(keys, net.parameters()):
#    print(k, "requires_grad = ", p.requires_grad)

In [15]:
EPOCH = 500
net.train()
for epoch in range(EPOCH):#EPOCH):
    print('epoch', epoch+1)    #epoch数の出力
    num = 0
    for (inputs, labels, input_img, input_mask, label_mask, img_mask) in trainloader:
    #for batch in trainloader:
        #num += 1
        #print(num)
        inputs, labels, input_img, input_mask, label_mask, img_mask = \
        inputs.to(device), labels.to(device), input_img.to(device),\
        input_mask.to(device), label_mask.to(device), img_mask.to(device)
        optimizer.zero_grad()
        handloc, pose_h, posedesc = net(input_img, inputs, input_mask, img_mask)
        #print("inputs", inputs[0])
        #print("labels", labels[0])
        #print("handloc", handloc[0])
        #print("pose_h", pose_h[0])
        #print("posedesc", posedesc[0])
        #手首位置基準座標→物体位置基準座標
        xd = torch.zeros_like(pose_h).to(device)
        yd = torch.zeros_like(pose_h).to(device)
        zd = torch.zeros_like(pose_h).to(device)
        for bsize in range(pose_h.size()[0]):
            xval = handloc[bsize][0].item()
            yval = handloc[bsize][1].item()
            zval = handloc[bsize][2].item()
            for i in range(int(pose_h.size()[1] / 3)):
                xd[bsize][i*3+0] = xval
                yd[bsize][i*3+1] = yval
                zd[bsize][i*3+2] = zval
        pose_o = (pose_h + xd + yd + zd).to(device)
        #print("pose_o", pose_o[0])
        loss = criterion(pose_o, labels, label_mask)#crit_outputs, crit_labels)
        #print("loss: ", loss)
        loss.backward()
        optimizer.step()

    sum_loss = 0.0          #lossの合計
    sum_total = 0           #dataの数の合計

    print("test_train")
    ttrain = 0
    #train dataを使ってテストをする(パラメータ更新がないようになっている)
    for (inputs, labels, input_img, input_mask, label_mask, img_mask) in trainloader:
        ttrain += 1
        inputs, labels, input_img, input_mask, label_mask, img_mask = \
        inputs.to(device), labels.to(device), input_img.to(device),\
        input_mask.to(device), label_mask.to(device), img_mask.to(device)
        optimizer.zero_grad()
        handloc, pose_h, posedesc = net(input_img, inputs, input_mask, img_mask)
        #手首位置基準座標→物体位置基準座標
        xd = torch.zeros_like(pose_h).to(device)
        yd = torch.zeros_like(pose_h).to(device)
        zd = torch.zeros_like(pose_h).to(device)
        for bsize in range(pose_h.size()[0]):
            xval = handloc[bsize][0].item()
            yval = handloc[bsize][1].item()
            zval = handloc[bsize][2].item()
            for i in range(int(pose_h.size()[1] / 3)):
                xd[bsize][i*3+0] = xval
                yd[bsize][i*3+1] = yval
                zd[bsize][i*3+2] = zval
        pose_o = (pose_h + xd + yd + zd).to(device)
        if ttrain == int(train_size / BATCH_SIZE):
            train_input_value.append(inputs)
            train_output_value.append(pose_o)
            train_desc_value.append(posedesc)
            train_handloc_value.append(handloc)
        loss = criterion(pose_o*100, labels*100, label_mask)
        sum_loss += loss.item()                            #lossを足していく
        sum_total += labels.size(0)                        #labelの数を足していくことでデータの総和を取る
    #print("len train dataset: ", len(trainloader.dataset))
    train_mean_loss = sum_loss*BATCH_SIZE/len(trainloader.dataset)
    print("train mean loss={}".format(train_mean_loss))  #loss出力
    train_loss_value.append(train_mean_loss)  #traindataのlossをグラフ描画のためにlistに保持
    if train_mean_loss > max_train_loss_value:
        max_train_loss_value = train_mean_loss

    sum_loss = 0.0
    sum_total = 0

    print("test_test")
    ttest = 0
    #test dataを使ってテストをする
    for (inputs, labels, input_img, input_mask, label_mask, img_mask) in testloader:
        ttest += 1
        inputs, labels, input_img, input_mask, label_mask, img_mask = \
        inputs.to(device), labels.to(device), input_img.to(device),\
        input_mask.to(device), label_mask.to(device), img_mask.to(device)
        optimizer.zero_grad()
        handloc, pose_h, posedesc = net(input_img, inputs, input_mask, img_mask)
        #手首位置基準座標→物体位置基準座標
        xd = torch.zeros_like(pose_h).to(device)
        yd = torch.zeros_like(pose_h).to(device)
        zd = torch.zeros_like(pose_h).to(device)
        for bsize in range(pose_h.size()[0]):
            xval = handloc[bsize][0].item()
            yval = handloc[bsize][1].item()
            zval = handloc[bsize][2].item()
            for i in range(int(pose_h.size()[1] / 3)):
                xd[bsize][i*3+0] = xval
                yd[bsize][i*3+1] = yval
                zd[bsize][i*3+2] = zval
        pose_o = (pose_h + xd + yd + zd).to(device)
        if ttest == int(test_size / BATCH_SIZE):
            test_input_value.append(inputs)
            test_output_value.append(pose_o)
            test_desc_value.append(posedesc)
            test_handloc_value.append(handloc)
        loss = criterion(pose_o*100, labels*100, label_mask)
        sum_loss += loss.item()
        sum_total += labels.size(0)
    #print("len test dataset: ", len(testloader.dataset))
    test_mean_loss = sum_loss*BATCH_SIZE/len(testloader.dataset)
    print("test mean loss={}".format(test_mean_loss))
    test_loss_value.append(test_mean_loss)
    if test_mean_loss > max_test_loss_value:
        max_test_loss_value = test_mean_loss
    if (min_test_loss_value != None and test_mean_loss < min_test_loss_value) or min_test_loss_value == None:
          min_test_loss_value = test_mean_loss
          torch.save(net.state_dict(), PATH + "\\model_epoch" + str(epoch+1))

epoch 1
test_train
train mean loss=80739.86979166667
test_test
test mean loss=119943.32470703125
epoch 2
test_train
train mean loss=79322.71152935606
test_test
test mean loss=119811.22119140625
epoch 3
test_train
train mean loss=78631.42589962122
test_test
test mean loss=120389.73095703125
epoch 4
test_train
train mean loss=78732.2431344697
test_test
test mean loss=119425.86572265625
epoch 5
test_train
train mean loss=78754.20857007576
test_test
test mean loss=119631.576171875
epoch 6
test_train
train mean loss=77259.17684659091
test_test
test mean loss=119103.9970703125
epoch 7
test_train
train mean loss=78214.64464962122
test_test
test mean loss=120405.630859375
epoch 8
test_train
train mean loss=77408.33901515152
test_test
test mean loss=119585.95361328125
epoch 9
test_train
train mean loss=77756.81403882576
test_test
test mean loss=120628.86181640625
epoch 10
test_train
train mean loss=78636.31273674243
test_test
test mean loss=120042.0146484375
epoch 11
test_train
train mean loss=

train mean loss=75864.1239938447
test_test
test mean loss=125235.3408203125
epoch 86
test_train
train mean loss=76076.69081439394
test_test
test mean loss=125458.56494140625
epoch 87
test_train
train mean loss=75602.91092566287
test_test
test mean loss=125769.748046875
epoch 88
test_train
train mean loss=76780.55835700757
test_test
test mean loss=126498.0556640625
epoch 89
test_train
train mean loss=76182.25609611743
test_test
test mean loss=126667.94775390625
epoch 90
test_train
train mean loss=75756.42223011363
test_test
test mean loss=126853.00146484375
epoch 91
test_train
train mean loss=74092.68110795454
test_test
test mean loss=126684.31591796875
epoch 92
test_train
train mean loss=74866.36103219698
test_test
test mean loss=125509.287109375
epoch 93
test_train
train mean loss=75569.82776988637
test_test
test mean loss=127050.970703125
epoch 94
test_train
train mean loss=74884.37819602272
test_test
test mean loss=126631.2080078125
epoch 95
test_train
train mean loss=75897.01236979

test mean loss=119832.5234375
epoch 169
test_train
train mean loss=72882.29060132576
test_test
test mean loss=119788.9873046875
epoch 170
test_train
train mean loss=72841.7138967803
test_test
test mean loss=120236.4794921875
epoch 171
test_train
train mean loss=73278.69673295454
test_test
test mean loss=120209.341796875
epoch 172
test_train
train mean loss=73610.25136126894
test_test
test mean loss=120173.55712890625
epoch 173
test_train
train mean loss=73142.83345170454
test_test
test mean loss=119950.36865234375
epoch 174
test_train
train mean loss=73284.81770833333
test_test
test mean loss=120471.3701171875
epoch 175
test_train
train mean loss=73315.89382102272
test_test
test mean loss=119446.2021484375
epoch 176
test_train
train mean loss=72939.82658617424
test_test
test mean loss=120284.88916015625
epoch 177
test_train
train mean loss=73220.11044034091
test_test
test mean loss=119651.57861328125
epoch 178
test_train
train mean loss=73672.88861268939
test_test
test mean loss=120252

test_train
train mean loss=73616.50124289772
test_test
test mean loss=120229.86181640625
epoch 253
test_train
train mean loss=73121.11393229167
test_test
test mean loss=120186.701171875
epoch 254
test_train
train mean loss=72676.4599905303
test_test
test mean loss=120878.095703125
epoch 255
test_train
train mean loss=73386.46839488637
test_test
test mean loss=121563.5712890625
epoch 256
test_train
train mean loss=73872.4179391572
test_test
test mean loss=120838.62451171875
epoch 257
test_train
train mean loss=73730.33528645833
test_test
test mean loss=121080.66552734375
epoch 258
test_train
train mean loss=73649.548828125
test_test
test mean loss=120798.13623046875
epoch 259
test_train
train mean loss=73636.68158143939
test_test
test mean loss=119376.73828125
epoch 260
test_train
train mean loss=73837.04746685606
test_test
test mean loss=120203.91650390625
epoch 261
test_train
train mean loss=73914.67424242424
test_test
test mean loss=120458.873046875
epoch 262
test_train
train mean lo

test mean loss=120523.52880859375
epoch 336
test_train
train mean loss=74078.33558238637
test_test
test mean loss=120920.23388671875
epoch 337
test_train
train mean loss=73419.45762310606
test_test
test mean loss=121070.2568359375
epoch 338
test_train
train mean loss=73158.49070785985
test_test
test mean loss=121281.009765625
epoch 339
test_train
train mean loss=73201.00467566287
test_test
test mean loss=121513.97705078125
epoch 340
test_train
train mean loss=73695.74289772728
test_test
test mean loss=120841.9404296875
epoch 341
test_train
train mean loss=73370.29509943182
test_test
test mean loss=120598.80810546875
epoch 342
test_train
train mean loss=73357.75390625
test_test
test mean loss=120481.60009765625
epoch 343
test_train
train mean loss=73072.79131155302
test_test
test mean loss=121028.77099609375
epoch 344
test_train
train mean loss=73822.62038352272
test_test
test mean loss=120894.51025390625
epoch 345
test_train
train mean loss=72675.27456202652
test_test
test mean loss=12

train mean loss=72174.81723484848
test_test
test mean loss=123652.689453125
epoch 420
test_train
train mean loss=72201.49976325757
test_test
test mean loss=123748.13671875
epoch 421
test_train
train mean loss=72703.52331912878
test_test
test mean loss=124344.87255859375
epoch 422
test_train
train mean loss=71379.90423768939
test_test
test mean loss=124141.9150390625
epoch 423
test_train
train mean loss=72700.21999289772
test_test
test mean loss=124270.8837890625
epoch 424
test_train
train mean loss=72096.11049952652
test_test
test mean loss=124066.46630859375
epoch 425
test_train
train mean loss=72254.76893939394
test_test
test mean loss=124291.55517578125
epoch 426
test_train
train mean loss=71897.6875
test_test
test mean loss=124075.447265625
epoch 427
test_train
train mean loss=71584.48307291667
test_test
test mean loss=123993.8857421875
epoch 428
test_train
train mean loss=72024.83132102272
test_test
test mean loss=123654.79345703125
epoch 429
test_train
train mean loss=72268.35996

RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 8.00 GiB total capacity; 5.03 GiB already allocated; 0 bytes free; 5.05 GiB reserved in total by PyTorch) (malloc at ..\c10\cuda\CUDACachingAllocator.cpp:289)
(no backtrace available)

In [110]:
#for k, p in zip(keys, net.parameters()):
#    print(k, "\n", p, "\n")

In [17]:
plt.figure(figsize=(7,7))      #グラフ描画用
ylim = max(max_train_loss_value, max_test_loss_value)

baseEPOCH = 100
num_epoch = 1
#act_num_epoch = baseEPOCH * num_epoch
act_num_epoch = 464

#以下グラフ描画
plt.plot(range(act_num_epoch), train_loss_value)
plt.plot(range(act_num_epoch), test_loss_value, c='#00ff00')
plt.xlim(0, act_num_epoch)
plt.ylim(0, ylim)
plt.xlabel('EPOCH')
plt.ylabel('LOSS(mm^2)')
plt.legend(['train loss', 'test loss'])
plt.title('loocation and pose LOSS')
plt.savefig(PATH + "\\loss_image_" + str(act_num_epoch) + ".png")
plt.clf()

<Figure size 504x504 with 0 Axes>

In [18]:
import matplotlib
#matplotlib.use('Agg')
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("darkgrid")

#結果画像出力
target_epoch = 463

HAND_PNT_NUM = 21

#fig = plt.figure(figsize=(6,6))
fig = plt.figure()
for i in range(len(train_input_value[target_epoch])):
    input_x = []
    input_y = []
    input_z = []
    mid = []
    output_x = []
    output_y = []
    output_z = []
    connect_x = []
    connect_y = []
    connect_z = []
    ax = fig.add_subplot(111, projection='3d')
    for j in range(int(len(train_input_value[target_epoch][0])/3)):
        input_x.append(train_input_value[target_epoch][i][j*3+0].item())
        input_y.append(-1*train_input_value[target_epoch][i][j*3+1].item())
        input_z.append(train_input_value[target_epoch][i][j*3+2].item())
        output_x.append(train_output_value[target_epoch][i][j*3+0].item())
        output_y.append(-1*train_output_value[target_epoch][i][j*3+1].item())
        output_z.append(train_output_value[target_epoch][i][j*3+2].item())
    x_max = max([max(input_x), max(output_x)], default = -10000)
    x_min = min([min(input_x), min(output_x)], default = 10000)
    y_max = max([max(input_y), max(output_y)], default = -10000)
    y_min = min([min(input_y), min(output_y)], default = 10000)
    z_max = max([max(input_z), max(output_z)], default = -10000)
    z_min = min([min(input_z), min(output_z)], default = 10000)
    
    #print(x_min, x_max, y_min, y_max, z_min, z_max, "(fixed: x_min, x_max, y_min, y_max, z_min, z_max)")
    
    #点が描画範囲内かどうか
    isInputPointsIn = [False] * HAND_PNT_NUM
    isOutputPointsIn = [False] * HAND_PNT_NUM
    #print(isPointsIn)
    
    for p in range(HAND_PNT_NUM):
        if x_min <= input_x[p] <= x_max and y_min <= input_y[p] <= y_max and z_min <= input_z[p] <= z_max:
            isInputPointsIn[p] = True
        if x_min <= output_x[p] <= x_max and y_min <= output_y[p] <= y_max and z_min <= output_z[p] <= z_max:
            isOutputPointsIn[p] = True
    
    #各点をプロット
    ax.scatter(input_x[0], input_y[0], zs=input_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(input_x[1], input_y[1], zs=input_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(input_x[2], input_y[2], zs=input_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(input_x[3], input_y[3], zs=input_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(input_x[4], input_y[4], zs=input_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(input_x[5], input_y[5], zs=input_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(input_x[6], input_y[6], zs=input_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(input_x[7], input_y[7], zs=input_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(input_x[8], input_y[8], zs=input_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(input_x[9], input_y[9], zs=input_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(input_x[10], input_y[10], zs=input_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(input_x[11], input_y[11], zs=input_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(input_x[12], input_y[12], zs=input_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(input_x[13], input_y[13], zs=input_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(input_x[14], input_y[14], zs=input_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(input_x[15], input_y[15], zs=input_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(input_x[16], input_y[16], zs=input_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(input_x[17], input_y[17], zs=input_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(input_x[18], input_y[18], zs=input_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(input_x[19], input_y[19], zs=input_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(input_x[20], input_y[20], zs=input_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isInputPointsIn[0] and isInputPointsIn[1]:
        ax.plot([input_x[0],input_x[1]], [input_y[0],input_y[1]], [input_z[0],input_z[1]], zdir='y', c='#cc0000')
    if isInputPointsIn[1] and isInputPointsIn[2]:
        ax.plot([input_x[1],input_x[2]], [input_y[1],input_y[2]], [input_z[1],input_z[2]], zdir='y', c='#b30000')
    if isInputPointsIn[2] and isInputPointsIn[3]:
        ax.plot([input_x[2],input_x[3]], [input_y[2],input_y[3]], [input_z[2],input_z[3]], zdir='y', c='#e60000')
    if isInputPointsIn[3] and isInputPointsIn[4]:
        ax.plot([input_x[3],input_x[4]], [input_y[3],input_y[4]], [input_z[3],input_z[4]], zdir='y', c='#ff0000')
    if isInputPointsIn[0] and isInputPointsIn[5]:
        ax.plot([input_x[0],input_x[5]], [input_y[0],input_y[5]], [input_z[0],input_z[5]], zdir='y', c='#a3cc00')
    if isInputPointsIn[5] and isInputPointsIn[6]:
        ax.plot([input_x[5],input_x[6]], [input_y[5],input_y[6]], [input_z[5],input_z[6]], zdir='y', c='#8fb300')
    if isInputPointsIn[6] and isInputPointsIn[7]:        
        ax.plot([input_x[6],input_x[7]], [input_y[6],input_y[7]], [input_z[6],input_z[7]], zdir='y', c='#b8e600')
    if isInputPointsIn[7] and isInputPointsIn[8]:
        ax.plot([input_x[7],input_x[8]], [input_y[7],input_y[8]], [input_z[7],input_z[8]], zdir='y', c='#ccff00')
    if isInputPointsIn[0] and isInputPointsIn[9]:
        ax.plot([input_x[0],input_x[9]], [input_y[0],input_y[9]], [input_z[0],input_z[9]], zdir='y', c='#00cc52')
    if isInputPointsIn[9] and isInputPointsIn[10]:
        ax.plot([input_x[9],input_x[10]], [input_y[9],input_y[10]], [input_z[9],input_z[10]], zdir='y', c='#00b347')
    if isInputPointsIn[10] and isInputPointsIn[11]:
        ax.plot([input_x[10],input_x[11]], [input_y[10],input_y[11]], [input_z[10],input_z[11]], zdir='y', c='#00e65c')
    if isInputPointsIn[11] and isInputPointsIn[12]:
        ax.plot([input_x[11],input_x[12]], [input_y[11],input_y[12]], [input_z[11],input_z[12]], zdir='y', c='#00ff66')
    if isInputPointsIn[0] and isInputPointsIn[13]:
        ax.plot([input_x[0],input_x[13]], [input_y[0],input_y[13]], [input_z[0],input_z[13]], zdir='y', c='#0052cc')
    if isInputPointsIn[13] and isInputPointsIn[14]:
        ax.plot([input_x[13],input_x[14]], [input_y[13],input_y[14]], [input_z[13],input_z[14]], zdir='y', c='#0047b3')
    if isInputPointsIn[14] and isInputPointsIn[15]:
        ax.plot([input_x[14],input_x[15]], [input_y[14],input_y[15]], [input_z[14],input_z[15]], zdir='y', c='#005ce6')
    if isInputPointsIn[15] and isInputPointsIn[16]:
        ax.plot([input_x[15],input_x[16]], [input_y[15],input_y[16]], [input_z[15],input_z[16]], zdir='y', c='#0066ff')
    if isInputPointsIn[0] and isInputPointsIn[17]:
        ax.plot([input_x[0],input_x[17]], [input_y[0],input_y[17]], [input_z[0],input_z[17]], zdir='y', c='#a300cc')
    if isInputPointsIn[17] and isInputPointsIn[18]:
        ax.plot([input_x[17],input_x[18]], [input_y[17],input_y[18]], [input_z[17],input_z[18]], zdir='y', c='#8f00b3')
    if isInputPointsIn[18] and isInputPointsIn[19]:
        ax.plot([input_x[18],input_x[19]], [input_y[18],input_y[19]], [input_z[18],input_z[19]], zdir='y', c='#b800e6')
    if isInputPointsIn[19] and isInputPointsIn[20]:
        ax.plot([input_x[19],input_x[20]], [input_y[19],input_y[20]], [input_z[19],input_z[20]], zdir='y', c='#cc00ff')
    
    ##stradrs = str(train_adrs[target_epoch][i].item()) #202011221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('input(train data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('input(train data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Input_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
    
    ax = fig.add_subplot(111, projection='3d')
    #各点をプロット
    ax.scatter(output_x[0], output_y[0], zs=output_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(output_x[1], output_y[1], zs=output_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(output_x[2], output_y[2], zs=output_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(output_x[3], output_y[3], zs=output_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(output_x[4], output_y[4], zs=output_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(output_x[5], output_y[5], zs=output_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(output_x[6], output_y[6], zs=output_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(output_x[7], output_y[7], zs=output_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(output_x[8], output_y[8], zs=output_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(output_x[9], output_y[9], zs=output_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(output_x[10], output_y[10], zs=output_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(output_x[11], output_y[11], zs=output_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(output_x[12], output_y[12], zs=output_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(output_x[13], output_y[13], zs=output_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(output_x[14], output_y[14], zs=output_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(output_x[15], output_y[15], zs=output_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(output_x[16], output_y[16], zs=output_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(output_x[17], output_y[17], zs=output_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(output_x[18], output_y[18], zs=output_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(output_x[19], output_y[19], zs=output_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(output_x[20], output_y[20], zs=output_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isOutputPointsIn[0] and isOutputPointsIn[1]:
        ax.plot([output_x[0],output_x[1]], [output_y[0],output_y[1]], [output_z[0],output_z[1]], zdir='y', c='#cc0000')
    if isOutputPointsIn[1] and isOutputPointsIn[2]:
        ax.plot([output_x[1],output_x[2]], [output_y[1],output_y[2]], [output_z[1],output_z[2]], zdir='y', c='#b30000')
    if isOutputPointsIn[2] and isOutputPointsIn[3]:
        ax.plot([output_x[2],output_x[3]], [output_y[2],output_y[3]], [output_z[2],output_z[3]], zdir='y', c='#e60000')
    if isOutputPointsIn[3] and isOutputPointsIn[4]:
        ax.plot([output_x[3],output_x[4]], [output_y[3],output_y[4]], [output_z[3],output_z[4]], zdir='y', c='#ff0000')
    if isOutputPointsIn[0] and isOutputPointsIn[5]:
        ax.plot([output_x[0],output_x[5]], [output_y[0],output_y[5]], [output_z[0],output_z[5]], zdir='y', c='#a3cc00')
    if isOutputPointsIn[5] and isOutputPointsIn[6]:
        ax.plot([output_x[5],output_x[6]], [output_y[5],output_y[6]], [output_z[5],output_z[6]], zdir='y', c='#8fb300')
    if isOutputPointsIn[6] and isOutputPointsIn[7]:        
        ax.plot([output_x[6],output_x[7]], [output_y[6],output_y[7]], [output_z[6],output_z[7]], zdir='y', c='#b8e600')
    if isOutputPointsIn[7] and isOutputPointsIn[8]:
        ax.plot([output_x[7],output_x[8]], [output_y[7],output_y[8]], [output_z[7],output_z[8]], zdir='y', c='#ccff00')
    if isOutputPointsIn[0] and isOutputPointsIn[9]:
        ax.plot([output_x[0],output_x[9]], [output_y[0],output_y[9]], [output_z[0],output_z[9]], zdir='y', c='#00cc52')
    if isOutputPointsIn[9] and isOutputPointsIn[10]:
        ax.plot([output_x[9],output_x[10]], [output_y[9],output_y[10]], [output_z[9],output_z[10]], zdir='y', c='#00b347')
    if isOutputPointsIn[10] and isOutputPointsIn[11]:
        ax.plot([output_x[10],output_x[11]], [output_y[10],output_y[11]], [output_z[10],output_z[11]], zdir='y', c='#00e65c')
    if isOutputPointsIn[11] and isOutputPointsIn[12]:
        ax.plot([output_x[11],output_x[12]], [output_y[11],output_y[12]], [output_z[11],output_z[12]], zdir='y', c='#00ff66')
    if isOutputPointsIn[0] and isOutputPointsIn[13]:
        ax.plot([output_x[0],output_x[13]], [output_y[0],output_y[13]], [output_z[0],output_z[13]], zdir='y', c='#0052cc')
    if isOutputPointsIn[13] and isOutputPointsIn[14]:
        ax.plot([output_x[13],output_x[14]], [output_y[13],output_y[14]], [output_z[13],output_z[14]], zdir='y', c='#0047b3')
    if isOutputPointsIn[14] and isOutputPointsIn[15]:
        ax.plot([output_x[14],output_x[15]], [output_y[14],output_y[15]], [output_z[14],output_z[15]], zdir='y', c='#005ce6')
    if isOutputPointsIn[15] and isOutputPointsIn[16]:
        ax.plot([output_x[15],output_x[16]], [output_y[15],output_y[16]], [output_z[15],output_z[16]], zdir='y', c='#0066ff')
    if isOutputPointsIn[0] and isOutputPointsIn[17]:
        ax.plot([output_x[0],output_x[17]], [output_y[0],output_y[17]], [output_z[0],output_z[17]], zdir='y', c='#a300cc')
    if isOutputPointsIn[17] and isOutputPointsIn[18]:
        ax.plot([output_x[17],output_x[18]], [output_y[17],output_y[18]], [output_z[17],output_z[18]], zdir='y', c='#8f00b3')
    if isOutputPointsIn[18] and isOutputPointsIn[19]:
        ax.plot([output_x[18],output_x[19]], [output_y[18],output_y[19]], [output_z[18],output_z[19]], zdir='y', c='#b800e6')
    if isOutputPointsIn[19] and isOutputPointsIn[20]:
        ax.plot([output_x[19],output_x[20]], [output_y[19],output_y[20]], [output_z[19],output_z[20]], zdir='y', c='#cc00ff')
    #ax.scatter(input_x, input_y, c='red', label = 'input')
    #ax.scatter(output_x, output_y, c='blue', label = 'output')
    
    #3stradrs = str(train_adrs[target_epoch][i].item()) #202011221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('output(train data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('output(train data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Output_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300,transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
for i in range(len(test_input_value[target_epoch])):
    input_x = []
    input_y = []
    input_z = []
    mid = []
    output_x = []
    output_y = []
    output_z = []
    connect_x = []
    connect_y = []
    connect_z = []
    ax = fig.add_subplot(111, projection='3d')
    for j in range(int(len(test_input_value[target_epoch][0])/3)):
        input_x.append(test_input_value[target_epoch][i][j*3+0].item())
        input_y.append(-1*test_input_value[target_epoch][i][j*3+1].item())
        input_z.append(test_input_value[target_epoch][i][j*3+2].item())
        output_x.append(test_output_value[target_epoch][i][j*3+0].item())
        output_y.append(-1*test_output_value[target_epoch][i][j*3+1].item())
        output_z.append(test_output_value[target_epoch][i][j*3+2].item())
    x_max = max([max(input_x), max(output_x)], default = -10000)
    x_min = min([min(input_x), min(output_x)], default = 10000)
    y_max = max([max(input_y), max(output_y)], default = -10000)
    y_min = min([min(input_y), min(output_y)], default = 10000)
    z_max = max([max(input_z), max(output_z)], default = -10000)
    z_min = min([min(input_z), min(output_z)], default = 10000)
    
    #print(x_min, x_max, y_min, y_max, z_min, z_max, "(fixed: x_min, x_max, y_min, y_max, z_min, z_max)")
    
    #点が描画範囲内かどうか
    isInputPointsIn = [False] * HAND_PNT_NUM
    isOutputPointsIn = [False] * HAND_PNT_NUM
    #print(isPointsIn)
    
    for p in range(HAND_PNT_NUM):
        if x_min <= input_x[p] <= x_max and y_min <= input_y[p] <= y_max and z_min <= input_z[p] <= z_max:
            isInputPointsIn[p] = True
        if x_min <= output_x[p] <= x_max and y_min <= output_y[p] <= y_max and z_min <= output_z[p] <= z_max:
            isOutputPointsIn[p] = True
    
    #各点をプロット
    ax.scatter(input_x[0], input_y[0], zs=input_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(input_x[1], input_y[1], zs=input_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(input_x[2], input_y[2], zs=input_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(input_x[3], input_y[3], zs=input_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(input_x[4], input_y[4], zs=input_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(input_x[5], input_y[5], zs=input_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(input_x[6], input_y[6], zs=input_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(input_x[7], input_y[7], zs=input_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(input_x[8], input_y[8], zs=input_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(input_x[9], input_y[9], zs=input_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(input_x[10], input_y[10], zs=input_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(input_x[11], input_y[11], zs=input_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(input_x[12], input_y[12], zs=input_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(input_x[13], input_y[13], zs=input_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(input_x[14], input_y[14], zs=input_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(input_x[15], input_y[15], zs=input_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(input_x[16], input_y[16], zs=input_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(input_x[17], input_y[17], zs=input_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(input_x[18], input_y[18], zs=input_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(input_x[19], input_y[19], zs=input_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(input_x[20], input_y[20], zs=input_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isInputPointsIn[0] and isInputPointsIn[1]:
        ax.plot([input_x[0],input_x[1]], [input_y[0],input_y[1]], [input_z[0],input_z[1]], zdir='y', c='#cc0000')
    if isInputPointsIn[1] and isInputPointsIn[2]:
        ax.plot([input_x[1],input_x[2]], [input_y[1],input_y[2]], [input_z[1],input_z[2]], zdir='y', c='#b30000')
    if isInputPointsIn[2] and isInputPointsIn[3]:
        ax.plot([input_x[2],input_x[3]], [input_y[2],input_y[3]], [input_z[2],input_z[3]], zdir='y', c='#e60000')
    if isInputPointsIn[3] and isInputPointsIn[4]:
        ax.plot([input_x[3],input_x[4]], [input_y[3],input_y[4]], [input_z[3],input_z[4]], zdir='y', c='#ff0000')
    if isInputPointsIn[0] and isInputPointsIn[5]:
        ax.plot([input_x[0],input_x[5]], [input_y[0],input_y[5]], [input_z[0],input_z[5]], zdir='y', c='#a3cc00')
    if isInputPointsIn[5] and isInputPointsIn[6]:
        ax.plot([input_x[5],input_x[6]], [input_y[5],input_y[6]], [input_z[5],input_z[6]], zdir='y', c='#8fb300')
    if isInputPointsIn[6] and isInputPointsIn[7]:        
        ax.plot([input_x[6],input_x[7]], [input_y[6],input_y[7]], [input_z[6],input_z[7]], zdir='y', c='#b8e600')
    if isInputPointsIn[7] and isInputPointsIn[8]:
        ax.plot([input_x[7],input_x[8]], [input_y[7],input_y[8]], [input_z[7],input_z[8]], zdir='y', c='#ccff00')
    if isInputPointsIn[0] and isInputPointsIn[9]:
        ax.plot([input_x[0],input_x[9]], [input_y[0],input_y[9]], [input_z[0],input_z[9]], zdir='y', c='#00cc52')
    if isInputPointsIn[9] and isInputPointsIn[10]:
        ax.plot([input_x[9],input_x[10]], [input_y[9],input_y[10]], [input_z[9],input_z[10]], zdir='y', c='#00b347')
    if isInputPointsIn[10] and isInputPointsIn[11]:
        ax.plot([input_x[10],input_x[11]], [input_y[10],input_y[11]], [input_z[10],input_z[11]], zdir='y', c='#00e65c')
    if isInputPointsIn[11] and isInputPointsIn[12]:
        ax.plot([input_x[11],input_x[12]], [input_y[11],input_y[12]], [input_z[11],input_z[12]], zdir='y', c='#00ff66')
    if isInputPointsIn[0] and isInputPointsIn[13]:
        ax.plot([input_x[0],input_x[13]], [input_y[0],input_y[13]], [input_z[0],input_z[13]], zdir='y', c='#0052cc')
    if isInputPointsIn[13] and isInputPointsIn[14]:
        ax.plot([input_x[13],input_x[14]], [input_y[13],input_y[14]], [input_z[13],input_z[14]], zdir='y', c='#0047b3')
    if isInputPointsIn[14] and isInputPointsIn[15]:
        ax.plot([input_x[14],input_x[15]], [input_y[14],input_y[15]], [input_z[14],input_z[15]], zdir='y', c='#005ce6')
    if isInputPointsIn[15] and isInputPointsIn[16]:
        ax.plot([input_x[15],input_x[16]], [input_y[15],input_y[16]], [input_z[15],input_z[16]], zdir='y', c='#0066ff')
    if isInputPointsIn[0] and isInputPointsIn[17]:
        ax.plot([input_x[0],input_x[17]], [input_y[0],input_y[17]], [input_z[0],input_z[17]], zdir='y', c='#a300cc')
    if isInputPointsIn[17] and isInputPointsIn[18]:
        ax.plot([input_x[17],input_x[18]], [input_y[17],input_y[18]], [input_z[17],input_z[18]], zdir='y', c='#8f00b3')
    if isInputPointsIn[18] and isInputPointsIn[19]:
        ax.plot([input_x[18],input_x[19]], [input_y[18],input_y[19]], [input_z[18],input_z[19]], zdir='y', c='#b800e6')
    if isInputPointsIn[19] and isInputPointsIn[20]:
        ax.plot([input_x[19],input_x[20]], [input_y[19],input_y[20]], [input_z[19],input_z[20]], zdir='y', c='#cc00ff')
    
    ##stradrs = str(test_adrs[target_epoch][i].item()) #202011221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('input(test data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('input(test data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Input_test_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
    
    ax = fig.add_subplot(111, projection='3d')
    #各点をプロット
    ax.scatter(output_x[0], output_y[0], zs=output_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(output_x[1], output_y[1], zs=output_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(output_x[2], output_y[2], zs=output_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(output_x[3], output_y[3], zs=output_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(output_x[4], output_y[4], zs=output_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(output_x[5], output_y[5], zs=output_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(output_x[6], output_y[6], zs=output_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(output_x[7], output_y[7], zs=output_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(output_x[8], output_y[8], zs=output_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(output_x[9], output_y[9], zs=output_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(output_x[10], output_y[10], zs=output_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(output_x[11], output_y[11], zs=output_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(output_x[12], output_y[12], zs=output_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(output_x[13], output_y[13], zs=output_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(output_x[14], output_y[14], zs=output_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(output_x[15], output_y[15], zs=output_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(output_x[16], output_y[16], zs=output_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(output_x[17], output_y[17], zs=output_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(output_x[18], output_y[18], zs=output_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(output_x[19], output_y[19], zs=output_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(output_x[20], output_y[20], zs=output_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isOutputPointsIn[0] and isOutputPointsIn[1]:
        ax.plot([output_x[0],output_x[1]], [output_y[0],output_y[1]], [output_z[0],output_z[1]], zdir='y', c='#cc0000')
    if isOutputPointsIn[1] and isOutputPointsIn[2]:
        ax.plot([output_x[1],output_x[2]], [output_y[1],output_y[2]], [output_z[1],output_z[2]], zdir='y', c='#b30000')
    if isOutputPointsIn[2] and isOutputPointsIn[3]:
        ax.plot([output_x[2],output_x[3]], [output_y[2],output_y[3]], [output_z[2],output_z[3]], zdir='y', c='#e60000')
    if isOutputPointsIn[3] and isOutputPointsIn[4]:
        ax.plot([output_x[3],output_x[4]], [output_y[3],output_y[4]], [output_z[3],output_z[4]], zdir='y', c='#ff0000')
    if isOutputPointsIn[0] and isOutputPointsIn[5]:
        ax.plot([output_x[0],output_x[5]], [output_y[0],output_y[5]], [output_z[0],output_z[5]], zdir='y', c='#a3cc00')
    if isOutputPointsIn[5] and isOutputPointsIn[6]:
        ax.plot([output_x[5],output_x[6]], [output_y[5],output_y[6]], [output_z[5],output_z[6]], zdir='y', c='#8fb300')
    if isOutputPointsIn[6] and isOutputPointsIn[7]:        
        ax.plot([output_x[6],output_x[7]], [output_y[6],output_y[7]], [output_z[6],output_z[7]], zdir='y', c='#b8e600')
    if isOutputPointsIn[7] and isOutputPointsIn[8]:
        ax.plot([output_x[7],output_x[8]], [output_y[7],output_y[8]], [output_z[7],output_z[8]], zdir='y', c='#ccff00')
    if isOutputPointsIn[0] and isOutputPointsIn[9]:
        ax.plot([output_x[0],output_x[9]], [output_y[0],output_y[9]], [output_z[0],output_z[9]], zdir='y', c='#00cc52')
    if isOutputPointsIn[9] and isOutputPointsIn[10]:
        ax.plot([output_x[9],output_x[10]], [output_y[9],output_y[10]], [output_z[9],output_z[10]], zdir='y', c='#00b347')
    if isOutputPointsIn[10] and isOutputPointsIn[11]:
        ax.plot([output_x[10],output_x[11]], [output_y[10],output_y[11]], [output_z[10],output_z[11]], zdir='y', c='#00e65c')
    if isOutputPointsIn[11] and isOutputPointsIn[12]:
        ax.plot([output_x[11],output_x[12]], [output_y[11],output_y[12]], [output_z[11],output_z[12]], zdir='y', c='#00ff66')
    if isOutputPointsIn[0] and isOutputPointsIn[13]:
        ax.plot([output_x[0],output_x[13]], [output_y[0],output_y[13]], [output_z[0],output_z[13]], zdir='y', c='#0052cc')
    if isOutputPointsIn[13] and isOutputPointsIn[14]:
        ax.plot([output_x[13],output_x[14]], [output_y[13],output_y[14]], [output_z[13],output_z[14]], zdir='y', c='#0047b3')
    if isOutputPointsIn[14] and isOutputPointsIn[15]:
        ax.plot([output_x[14],output_x[15]], [output_y[14],output_y[15]], [output_z[14],output_z[15]], zdir='y', c='#005ce6')
    if isOutputPointsIn[15] and isOutputPointsIn[16]:
        ax.plot([output_x[15],output_x[16]], [output_y[15],output_y[16]], [output_z[15],output_z[16]], zdir='y', c='#0066ff')
    if isOutputPointsIn[0] and isOutputPointsIn[17]:
        ax.plot([output_x[0],output_x[17]], [output_y[0],output_y[17]], [output_z[0],output_z[17]], zdir='y', c='#a300cc')
    if isOutputPointsIn[17] and isOutputPointsIn[18]:
        ax.plot([output_x[17],output_x[18]], [output_y[17],output_y[18]], [output_z[17],output_z[18]], zdir='y', c='#8f00b3')
    if isOutputPointsIn[18] and isOutputPointsIn[19]:
        ax.plot([output_x[18],output_x[19]], [output_y[18],output_y[19]], [output_z[18],output_z[19]], zdir='y', c='#b800e6')
    if isOutputPointsIn[19] and isOutputPointsIn[20]:
        ax.plot([output_x[19],output_x[20]], [output_y[19],output_y[20]], [output_z[19],output_z[20]], zdir='y', c='#cc00ff')
    #ax.scatter(input_x, input_y, c='red', label = 'input')
    #ax.scatter(output_x, output_y, c='blue', label = 'output')
    
    ##stradrs = str(test_adrs[target_epoch][i].item()) #11221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('output(test data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('output(test data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Output_test_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
plt.close()

In [19]:
#出力値保存

with open(PATH + "\\outputs\\network_outs.csv", mode='w') as f:
    writer = csv.writer(f)
    writer.writerow(["train_test"," "])
    for i in range(len(train_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(i+1)," "])
        for j in range(len(train_input_value[i])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            input_z = []
            output_x = []
            output_y = []
            output_z = []
            descriptor = []
            handlocation = []
            for k in range(int(len(train_input_value[i][j])/4)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(train_input_value[i][j][k*4+0].item())
                input_y.append(train_input_value[i][j][k*4+1].item())
                input_z.append(train_input_value[i][j][k*4+2].item())
                output_x.append(train_output_value[i][j][k*3+0].item())
                output_y.append(train_output_value[i][j][k*3+1].item())
                output_z.append(train_output_value[i][j][k*3+2].item())
            for l in range(len(train_desc_value[i][j])):
                descriptor.append(train_desc_value[i][j][l].item())
            for l in range(len(train_handloc_value[i][j])):
                handlocation.append(train_handloc_value[i][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(descriptor)
            writer.writerow(handlocation)
    writer.writerow(["test_test"," "])
    for i in range(len(test_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(i+1)," "])
        for j in range(len(test_input_value[i])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            input_z = []
            output_x = []
            output_y = []
            output_z = []
            descriptor = []
            handlocation = []
            for k in range(int(len(test_input_value[i][j])/4)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(test_input_value[i][j][k*4+0].item())
                input_y.append(test_input_value[i][j][k*4+1].item())
                input_z.append(test_input_value[i][j][k*4+2].item())
                output_x.append(test_output_value[i][j][k*3+0].item())
                output_y.append(test_output_value[i][j][k*3+1].item())
                output_z.append(test_output_value[i][j][k*3+2].item())
            for l in range(len(test_desc_value[i][j])):
                descriptor.append(test_desc_value[i][j][l].item())
            for l in range(len(test_handloc_value[i][j])):
                handlocation.append(test_handloc_value[i][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(descriptor)
            writer.writerow(handlocation)
    print("fin save.")

fin save.


In [20]:
sean_firsts = []
for sean in train_seanset:
    #sean_imgcfile = glob.glob(sean + "\\object*.png")[0]
    sean_imgdfile = glob.glob(sean + "\\objectd*.png")[0]
    s_datafile = glob.glob(sean + "\\data\\*0.csv")[0]
    #s_labelfile = glob.glob(sean + "\\label\\*.csv")[0]
    pair = []
    pair.append(s_datafile)
    #pair.append(s_labelfile)
    pair.append(sean_imgdfile)
    sean_firsts.append(pair)

data_recall = []
img_recall = []
mask_recall = []
imgmask_recall = []
for i in range(len(sean_firsts)):
    # 画像読み込み
    image = Image.open(sean_firsts[i][1])
    # グレイスケール変換
    #image = image.convert('L')
    # リサイズ
    image = image.resize((image_size, image_size))
    ## 画像から配列に変換
    #img_recall.append(np.asarray(image))
    # 画像から配列に変換
    img_array = np.asarray(image)
    img_recall.append(img_array)
    img_mask_array = np.zeros((image_size, image_size), np.uint8) #画像マスク
    
    #元画像の画素値が0の部分のみマスク画像の画素値を1にする
    for h in range(img_array.shape[0]):
        for w in range(img_array.shape[1]):
            if 0 <= img_array[h,w] < 500:
                img_mask_array[h,w] = 1
    #img_names.append(os.path.basename(imgfile))
    #file_split = [i for i in file.split('_')]
    imgmask_recall.append(img_mask_array)
    
    data_points = []
    data_masks = []
    with open(sean_firsts[i][0]) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                for point in row:
                    if int(point) == -10000:
                        data_points.append(float(0))
                        data_masks.append(0)
                    else:
                        data_points.append(float(point))
                        data_masks.append(1)
                data_points = np.asarray(data_points)
                data_masks = np.asarray(data_masks)
                num += 1
    data_recall.append(np.asarray(data_points))
    mask_recall.append(np.asarray(data_masks))

print("fin loading")

data_recall = np.array(data_recall).astype('float32')
img_recall = np.array(img_recall).astype('float32')/1000
mask_recall = np.array(mask_recall).astype('float32')
imgmask_recall = np.array(imgmask_recall).astype('float32')

print("data",data_recall[0])
print("img",img_recall[0])
print("dmask",mask_recall[0])
print("imgmask",imgmask_recall[0])

fin loading
data [-267. -214.  151.    0.    0.    0. -229. -102.  -87. -228.  -74.  -66.
    0.    0.    0. -247. -151.  192. -249. -108.  208. -224.  -48.   -8.
 -225.  -33.  -62. -247. -145.  180.    0.    0.    0. -225.  -42.  -62.
 -234.  -34.  -73. -250. -145.  180. -224.  -73.  -36. -231.  -49.  -79.
 -239.  -40.  -96. -225.  -97.  -60. -228.  -74.  -66. -235.  -56.  -96.
 -238.  -51. -105.    0.]
img [[3.425 3.425 3.429 ... 3.441 3.441 3.441]
 [3.416 3.416 3.416 ... 3.418 3.418 3.419]
 [3.415 3.415 3.412 ... 3.399 3.399 3.396]
 ...
 [3.028 3.028 3.024 ... 2.922 2.922 2.954]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]]
dmask [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
imgmask [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0

In [21]:
#trans = torchvision.transforms.Compose(
#    [torchvision.transforms.ToTensor()]#,torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
#)

class Mydatasets2(torch.utils.data.Dataset):
    def __init__(self, datas, img_array, data_masks, img_masks, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.img_array = img_array
        self.masks = data_masks
        self.img_masks = img_masks

        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data
        i_img = self.img_array
        i_mask = self.masks
        i_imgmask = self.img_masks
        
        #print(out_label)
        #print(type(i_label))
        out_data = np.array(i_data.astype(np.float32))
        out_img = np.array(i_img.astype(np.float32))
        out_mask = np.array(i_mask.astype(np.float32))
        out_imgmask = np.array(i_imgmask.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            #out_data = self.transform(i_data)
            out_img = self.transform(out_img)
            out_imgmask = self.transform(out_imgmask)
            #out_label = self.transform2(out_label)
        
        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)

        return out_data, out_img, out_mask, out_imgmask

In [23]:
#recallset = Mydatasets2(datas = data_recall, img_array = img_recall, data_masks = mask_recall, transform = trans)

def recall_sequence(data, mask, img, imask):
    data = data.astype('float32')/100
    mask = mask.astype('float32')
    img = img.astype('float32')
    imask = imask.astype('float32')
    
    #data = np.array(data.astype(np.float32))
    #mask = np.array(mask.astype(np.float32))
    #img = trans(np.array(img.astype(np.float32)))
    
    print("data:",data)
    print("mask:",mask)
    print("img:",img)
    print("imask: ",imask)
    #init_l_points = np.array(init_l_points.astype(np.float32))
    #print(init_l_points)
    recallset = Mydatasets2(datas = data, img_array = img, data_masks = mask, img_masks = imask, transform = trans)
    print("recall_set:",recallset)
    recallloader = torch.utils.data.DataLoader(recallset, batch_size = 15, shuffle = False, num_workers = 0)
    
    #net.eval()
    for (inputs, imgs, masks, imasks) in recallloader:
    #tmp = recallloader.__iter__()
    #inputs, imgs, masks = tmp.next()
        inputs, imgs, masks, imasks = inputs.to(device), imgs.to(device), masks.to(device), imasks.to(device)
        optimizer.zero_grad()
        handloc, pose_h, posedesc = net(input_img, inputs, input_mask, img_mask)
        #手首位置基準座標→物体位置基準座標
        xd = torch.zeros_like(pose_h).to(device)
        yd = torch.zeros_like(pose_h).to(device)
        zd = torch.zeros_like(pose_h).to(device)
        for bsize in range(pose_h.size()[0]):
            xval = handloc[bsize][0].item()
            yval = handloc[bsize][1].item()
            zval = handloc[bsize][2].item()
            for i in range(int(pose_h.size()[1] / 3)):
                xd[bsize][i*3+0] = xval
                yd[bsize][i*3+1] = yval
                zd[bsize][i*3+2] = zval
        recall_out = (pose_h + xd + yd + zd).to(device)
        print(recall_out)
        break
    recall_out_np = recall_out.to('cpu').detach().numpy().copy()
    print(recall_out_np[0])
    return recall_out_np[0]

mask_static = np.ones(data_recall.shape[1])

for r in range(len(data_recall)):
    with open(PATH + "\\recalls\\{:0=4}".format(r) + "\\recall_0.csv" , 'w') as f:
        writer = csv.writer(f)
        record = data_recall[r]
        writer.writerow(record.tolist())
    initial = recall_sequence(data_recall[r], mask_recall[r], img_recall[r], imgmask_recall[r])
    print("init:",initial)
    print("type:",type(initial))
    
    for t in range(20):
        initial = (initial*100).tolist()
        initial.append(t+1)
        with open(PATH + "\\recalls\\{:0=4}".format(r) + "\\recall_" + str(t+1) + ".csv", 'w') as f:
            writer = csv.writer(f)
            writer.writerow(initial)
        initial = recall_sequence( np.array(initial), mask_static, img_recall[r], imgmask_recall[r])
        
        
        

data: [-2.67 -2.14  1.51  0.    0.    0.   -2.29 -1.02 -0.87 -2.28 -0.74 -0.66
  0.    0.    0.   -2.47 -1.51  1.92 -2.49 -1.08  2.08 -2.24 -0.48 -0.08
 -2.25 -0.33 -0.62 -2.47 -1.45  1.8   0.    0.    0.   -2.25 -0.42 -0.62
 -2.34 -0.34 -0.73 -2.5  -1.45  1.8  -2.24 -0.73 -0.36 -2.31 -0.49 -0.79
 -2.39 -0.4  -0.96 -2.25 -0.97 -0.6  -2.28 -0.74 -0.66 -2.35 -0.56 -0.96
 -2.38 -0.51 -1.05  0.  ]
mask: [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
img: [[3.425 3.425 3.429 ... 3.441 3.441 3.441]
 [3.416 3.416 3.416 ... 3.418 3.418 3.419]
 [3.415 3.415 3.412 ... 3.399 3.399 3.396]
 ...
 [3.028 3.028 3.024 ... 2.922 2.922 2.954]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 8.00 GiB total capacity; 5.02 GiB already allocated; 0 bytes free; 5.03 GiB reserved in total by PyTorch)

In [135]:
class Mydatasets3(torch.utils.data.Dataset):
    def __init__(self, datas, img_array, data_masks, img_masks, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.img_array = img_array
        self.masks = data_masks
        self.img_masks = img_masks

        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data[idx]
        i_img = self.img_array[idx]
        i_mask = self.masks[idx]
        i_imask = self.img_masks[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_data = np.array(i_data.astype(np.float32))
        out_img = np.array(i_img.astype(np.float32))
        out_mask = np.array(i_mask.astype(np.float32))
        out_imask = np.array(i_imask.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            #out_data = self.transform(i_data)
            out_img = self.transform(out_img)
            out_imask = self.transform(out_imask)
            #out_label = self.transform2(out_label)
        
        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)

        return out_data, out_img, out_mask, out_imask

In [137]:
data_check = []
img_check = []
mask_check = []
imgmask_check = []
for i in range(len(img_skeleton_sets_train)):
    data_check.append(img_skeleton_sets_train[i][0])
    img_check.append(img_skeleton_sets_train[i][1])
    mask_check.append(img_skeleton_sets_train[i][2])
    imgmask_check.append(img_skeleton_sets_train[i][3])
    
data_check = np.array(data_check).astype('float32')/100
img_check = np.array(img_check).astype('float32')/1000
mask_check = np.array(mask_check).astype('float32')
imgmask_check = np.array(imgmask_check).astype('float32')
print(data_check.shape)
print(img_check.shape)
print(mask_check.shape)
print(imgmask_check.shape)

checkset = Mydatasets3(datas = data_check, img_array = img_check, data_masks = mask_check, img_masks = imgmask_check, transform = trans)

checkloader = torch.utils.data.DataLoader(checkset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)

print("train_test")
#train dataを使ってテストをする(パラメータ更新がないようになっている)
num = 0
net.eval()
for (inputs, input_img, input_mask, input_imask) in checkloader:
    inputs, input_img, input_mask, input_imask = \
    inputs.to(device), input_img.to(device), input_mask.to(device), input_imask.to(device)
    optimizer.zero_grad()
    outputs = net(input_img, inputs, input_mask, input_imask)
    out_np = (outputs.to('cpu').detach().numpy().copy()) * 100
    for b in range(BATCH_SIZE):
        with open(PATH + "\\checks\\check_" + str((num * BATCH_SIZE) + b) + ".csv" , 'w') as f:
            writer = csv.writer(f)
            writer.writerow(out_np[b].tolist())
    num += 1

(600, 64)
(600, 36, 36)
(600, 64)
(600, 36, 36)
train_test


In [138]:
torch.save(net.state_dict(), PATH + "\\model8")

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.pool = nn.AvgPool2d(2, stride=2)

        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        self.bn2d1 = nn.BatchNorm2d(16)
        self.bn2d2 = nn.BatchNorm2d(32)
        
        self.fcm = Ignore(32 * 7 * 7 + 64, 32 * 7 * 7 + 64)
        self.bnm = nn.BatchNorm1d(32 * 7 * 7 + 64)
        #fully connect for hand Location(x,y,z)
        self.fcL1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcL2 = nn.Linear(300, 30)
        self.fcL3 = nn.Linear(30, 3)        
        self.bnL1 = nn.BatchNorm1d(300)
        self.bnL2 = nn.BatchNorm1d(30)
        #fully connect for hand Pose Descriptor(8 properties)
        self.fcPD1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcPD2 = nn.Linear(300, 60)
        self.fcPD3 = nn.Linear(60, 8)
        self.bnPD1 = nn.BatchNorm1d(300)
        self.bnPD2 = nn.BatchNorm1d(60)

    def forward(self, x, y, m, im):
        #print("original: ",x[0][0:2],"\n",x[1][0:2], x.size())
        #print("original mask: ",im[0][0:2],"\n",im[1][0:2], im.size())
        
        #Conv2d 1
        x = self.conv1(x)
        im = self.conv1(im)
        #print("conv1: ",x[0:2][0:2], x.size())
        #print("conv1 mask: ",im[0:2][0:2], im.size())
        im_conv1 = im.clone().detach().to(torch.device("cuda:0"))
        for c in range(im.size()[1]):
            mode, ind = torch.mode(torch.reshape(im[0][c].clone().detach(), (-1, im.size()[2] * im.size()[3])))
            im_conv1[:,c] = torch.sub(im_conv1[:,c], mode.item())
        #print("conv1 mask fix: ",im_conv1[0:2][0:2], im_conv1.size())       
        
        #ReLU→BatchNorm2d 1
        x = self.bn2d1(self.relu(x))
        #print("relu1: ", x[0:2][0:2], x.size())
        #x = self.bn2d1(self.tanh(x))
        #print("tanh1: ", x[0:2][0:2], x.size())
        
        #AvgPool2d
        x = self.pool(x)
        im = self.pool(im_conv1)
        #print("pool1: ", x[0:2][0:2], x.size())
        #print("pool1 mask: ",im[0:2][0:2], im.size())
        
        #Conv2d 2
        x = self.conv2(x)
        im = self.conv2(im)
        #print("conv2: ",x[0:2][0:2], x.size())
        #print("conv2 mask: ",im[0:2][0:2], im.size())
        im_conv2 = im.clone().detach().to(torch.device("cuda:0"))
        for c in range(im.size()[1]):
            mode, ind = torch.mode(torch.reshape(im[0][c].clone().detach(), (-1, im.size()[2] * im.size()[3])))
            im_conv2[:,c] = torch.sub(im_conv2[:,c], mode.item())
        #print("conv2 mask fix: ",im_conv2[0:2][0:2], im_conv2.size())
        
        #ReLU→BatchNorm2d 2
        x = self.bn2d2(self.relu(x))
        #print("relu2: ", x[0:2][0:2], x.size())
        #x = self.bn2d2(self.tanh(x))
        #print("tanh2: ", x[0:2][0:2], x.size())
        
        #AvgPool2d
        x = self.pool(x)
        im = self.pool(im_conv2)
        #print("pool2: ", x[0:2][0:2], x.size())
        #print("pool2 mask: ",im[0:2][0:2], im.size())
        
        #1次元ベクトル化
        x = x.view(x.size()[0], -1)
        im = im.view(im.size()[0], -1)
        #print("view: ", x[0:2], x.size())
        #print("view mask: ",im[0:2], im.size())
        
        #マスク再構築(0→1, 0以外→0)
        im = torch.logical_not(torch.logical_and(im, torch.tensor([True]).to(torch.device("cuda:0")))).float()
        #print("mask_img: ", im[0:2], im.size())
        
        #骨格データと結合
        x = torch.cat([x, y], axis = -1)
        m = torch.cat([im, m], axis = -1)
        #print("cat: ", x[0:2], x.size())
        #print("cat mask: ", m[0:2], m.size())
        
        #MaskedLinear
        x = self.fcm(x, m)
        #print("masked: ", x[0], x.size())
        x = self.bnm(self.relu(x))
        #x = self.bnm(self.tanh(x))
        xL = self.fcL1(x)
        xPD = self.fcPD1(x)
        #print("fc1: ",x[0])
        xL = self.bnL1(self.relu(xL))
        xPD = self.bnPD1(self.relu(xPD))
        #x = self.bn1(self.tanh(x))
        xL = self.fcL2(xL)
        xPD = self.fcPD2(xPD)
        #print("fc2: ",x[0])
        xL = self.bnL2(self.relu(xL))
        xPD = self.bnPD2(self.relu(xPD))
        #x = self.bn2(self.tanh(x))
        xL = self.fcL3(xL)
        xPD = self.fcPD3(xPD)
        return xL, xPD